In [ ]:
import numpy as np
import itertools 
import math
import unicodedata as uc

In [ ]:
def clean_data(lines):
  cleaned_data = []
  for line in lines:
    not_included = 0
    cleaned_line = []
    for i in range(len(line)):
      if (ord(line[i])<2304 or ord(line[i])>2431) and line[i] != ' ' and line[i] != '\n': # remove new line character and english/roman characters
          not_included = 1
          break
      elif line[i] == ' ' or line[i] == '\n' or line[i] == u'\u0964':
        cleaned_line.append(' ')
      else :
        cleaned_line.append(line[i])
    if not_included ==1 :
      continue
    cleaned_line = ''.join(cleaned_line)
    cleaned_line = ' '.join(cleaned_line.split())
    cleaned_data.append(cleaned_line)
  return cleaned_data

In [ ]:
def clean_sentence(line):
  not_included = 0
  cleaned_line = []
  for i in range(len(line)):
    if (ord(line[i])<2304 or ord(line[i])>2431) and line[i] != ' ' and line[i] != '\n': # remove new line character and english/roman characters
        continue
    elif line[i] == ' ' or line[i] == '\n' or line[i] == u'\u0964':
      cleaned_line.append(' ')
    else :
      cleaned_line.append(line[i])
  cleaned_line = ''.join(cleaned_line)
  cleaned_line = ' '.join(cleaned_line.split())
  return cleaned_line

In [ ]:
def bigram(c) :
  freq={}
  count={}
  dictionary = set()
  for line in c:
    for i in range(len(line)):
      if line[i] in count:
        count[line[i]]+=1
      else:
        count[line[i]]=1
        dictionary.add(line[i])
      if i < len(line)-1:
        if (line[i],line[i+1]) in freq:
          freq[(line[i],line[i+1])]+=1
        else:
          freq[(line[i],line[i+1])]=1
      if i < len(line)-2:
        if (line[i],line[i+2]) in freq:
          freq[(line[i],line[i+2])]+=1
        else:
          freq[(line[i],line[i+2])]=1
  dictionary = list(dictionary)
  return freq,count,dictionary

In [ ]:
def score(bifreq,count,line,ind,similar):
  scores={}
  for s in similar:
    score=0.0
    if ind>0 and s in count and line[ind-1] in count:
      t=0.0
      if (s,line[ind-1]) in bifreq:
        t+=bifreq[(s,line[ind-1])]
      if (line[ind-1],s) in bifreq:
        t+=bifreq[(line[ind-1],s)]
      score+=t/math.sqrt(count[s]*count[line[ind-1]])
    if ind>1 and s in count and line[ind-2] in count:
      t=0.0
      if (s,line[ind-2]) in bifreq:
        t+=bifreq[(s,line[ind-2])]
      if (line[ind-2],s) in bifreq:
        t+=bifreq[(line[ind-2],s)]
      score+=t/math.sqrt(count[s]*count[line[ind-2]])
    if ind>2 and s in count and line[ind-3] in count:
      t=0.0
      if (s,line[ind-3]) in bifreq:
        t+=bifreq[(s,line[ind-3])]
      if (line[ind-3],s) in bifreq:
        t+=bifreq[(line[ind-3],s)]
      score+=t/math.sqrt(count[s]*count[line[ind-3]])
    if ind<len(line)-1 and s in count and line[ind+1] in count:
      t=0.0
      if (s,line[ind+1]) in bifreq:
        t+=bifreq[(s,line[ind+1])]
      if (line[ind+1],s) in bifreq:
        t+=bifreq[(line[ind+1],s)]
      score+=t/math.sqrt(count[s]*count[line[ind+1]])
    if ind<len(line)-2 and s in count and line[ind+2] in count:
      t=0.0
      if (s,line[ind+2]) in bifreq:
        t+=bifreq[(s,line[ind+2])]
      if (line[ind+2],s) in bifreq:
        t+=bifreq[(line[ind+2],s)]
      score+=t/math.sqrt(count[s]*count[line[ind+2]])
    if ind<len(line)-3 and s in count and line[ind+3] in count:
      t=0.0
      if (s,line[ind+3]) in bifreq:
        t+=bifreq[(s,line[ind+3])]
      if (line[ind+3],s) in bifreq:
        t+=bifreq[(line[ind+3],s)]
      score+=t/math.sqrt(count[s]*count[line[ind+3]])
    scores[s]=score
  ret = dict(sorted(scores.items(),key = lambda x:x[1],reverse = True))
  return ret

In [ ]:
characters = [u"ँ", u"ं", u"ः", u"ऄ", u"अ", u"आ", u"इ", u"ई", u"उ", u"ऊ", u"ऋ",
              u"ऌ", u"ऍ", u"ऎ", u"ए", u"ऐ", u"ऑ", u"ऒ", u"ओ", u"औ", u"क", u"ख",
              u"ग", u"घ", u"ङ", u"च", u"छ", u"ज", u"झ", u"ञ", u"ट", u"ठ", u"ड",
              u"ढ", u"ण", u"त", u"थ", u"द", u"ध", u"न", u"ऩ", u"प", u"फ", u"ब",
              u"भ", u"म", u"य", u"र", u"ऱ", u"ल", u"ळ", u"ऴ", u"व", u"श", u"ष",
              u"स", u"ह", u"ऺ", u"ऻ", u"़", u"ऽ", u"ा", u"ि", u"ी", u"ु", u"ू",
              u"ृ", u"ॄ", u"ॅ", u"ॆ", u"े", u"ै", u"ॉ", u"ॊ", u"ो", u"ौ", u"्",
              u"ॎ", u"ॏ", u"ॐ", u"॑", u"॒", u"॓", u"॔", u"ॕ", u"ॖ", u"ॗ", chr(2392),
              chr(2393), chr(2394), chr(2395), chr(2396), chr(2397), chr(2398), chr(2399), u"ॠ", u"ॡ", u"ॢ", u"ॣ",
              u"।", u"॥", u"०", u"१", u"२", u"३", u"४", u"५", u"६", u"७", u"८",
              u"९", u"॰", u"ॱ", u"ॲ", u"ॳ", u"ॴ", u"ॵ", u"ॶ", u"ॷ", u"ॸ", u"ॹ",
              u"ॺ", u"ॻ", u"ॼ", u"ॽ", u"ॾ", u"ॿ"]

mapping = ['0', 'N', '0', '0', 'A', 'A', 'B', 'B', 'C', 'C', 'P', 'Q',
          '0', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F',
          'G', 'H', 'H', 'H', 'H', 'G', 'I', 'I', 'I', 'I', 'J', 'K',
          'K', 'K', 'K', 'L', 'L', 'M', 'M', 'M', 'M', 'N', 'O', 'P',
          'P', 'Q', 'Q', 'Q', 'R', 'S', 'S', 'S', 'T', '0', '0', '0',
          '0', 'A', 'B', 'B', 'C', 'C', 'P', 'P', 'E', 'D', 'D', 'D',
          'D', 'E', 'E', 'E', '0', '0', '0', '0', '0', '0', '0', '0',
          '0', '0', 'E', '0', '0', '0', '0', '0', '0', '0', '0', 'P',
          'Q', 'Q', 'Q', '0', '0', '0', '1', '2', '3', '4', '5', '6',
          '7', '8', '9', '0', '0', '0', '0', '0', '0', '0', '0', '0',
          '0', 'J', 'J', 'Q', 'P', 'P', 'F']
char_to_phonetic = {}
for i in range(len(characters)):
  char_to_phonetic[characters[i]] = mapping[i]

In [ ]:
def calc_phon_score(word):
  score = ""
  for letter in word :
    score += char_to_phonetic[letter]
  return score
def get_words_by_phonetic(dictionary):
  phonetic_to_words = {}
  for word in dictionary:
    score = calc_phon_score(word)
    if score in phonetic_to_words:
      phonetic_to_words[score].append(word)
    else :
      phonetic_to_words[score] = []
      phonetic_to_words[score].append(word)
  return phonetic_to_words

In [ ]:
#Setting up the basic character list 
vow = [ 'अ','आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'अं', 'अ:']
matra = ['','ा','ि','ी','ु','ू','े','ै','ो','ौ','ं','ः']
vowels = ['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'ऋ', 'अं', 'अ:']
con = [ 'क', 'ख', 'ग', 'घ', 'ड़', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
       'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह','ढ़','ड़']
matra_to_vow = {}
vow_to_matra = {}
for i in range(len(matra)):
    matra_to_vow[matra[i]]=vow[i]
    vow_to_matra[vow[i]]=matra[i]


In [ ]:
# This function corrects one item : word that is provided to it.
def correct_item(s):
    ans = []
    halant = u'\N{DEVANAGARI SIGN VIRAMA}'
    _a = u'अ'
    for i in range(len(s)-1):
        c = s[i]
        cat = uc.category(c)[0]
        if cat == 'Z':
            ans.append(c)
        elif c in vowels :
            ans.append(c)
        elif cat == 'L' and s[i+1] == halant:
            # if a consonant is already followed by a halant
            ans.append(str(c)+halant)
        elif cat == 'L' and not (uc.category(s[i+1])[0]=='M'):
            # if a consonant is not by a marker already, like a matra or something
            ans.append(str(c)+halant)
            ans.append(_a)
        elif cat == 'M':
            # M denotes matras or halants -> cant make a character starting with them
            continue
        elif cat == 'N' or cat == 'P':
            # a number or a puntuation has to be kept as is 
            ans.append(c)
        else:
            # if a consonant is followed by a marker, i.e. in this case a matra
            ans.append(str(c)+halant)
            ans.append(str(s[i+1]))
    
    last = s[len(s)-1]
    cat = uc.category(last)[0]
    if cat == 'N' :
        ans.append(str(last))
    if cat == 'L' and last in vowels:
        ans.append(str(last))
    if cat == 'L' and last in con:
        ans.append(str(last)+halant)
        ans.append(_a)
    # a list of strings has been created for each sentence
    # converting the vowels present as matras into vowels
    i=0
    for c in ans:
        if c in matra:
            ans[i] = matra_to_vow[c]
        i+=1
    return ans

In [ ]:
def edit_distance(s_a,s_b):
    # numpy array not used because the words cant be very big.
    a = correct_item(s_a)
    b = correct_item(s_b)
    large = 1000
    insert = 1;
    remove = 1;
    replace = 1;
    if abs(len(a)-len(b)) > 4 :
        return large
    (rows,cols) = (len(a)+1, len(b)+1)
    dp = [[0 for i in range(cols)] for j in range(rows)]
    for i in range(1,rows):
        dp[i][0] = i*insert
    for i in range(1,cols):
        dp[0][i] = i*remove

    for i in range(1,rows):
        for j in range(1,cols):
            if a[i-1]==b[j-1]: 
                dp[i][j] = dp[i-1][j-1]
            else :
                l = [dp[i][j-1] + insert, dp[i][j-1] + remove, dp[i-1][j-1] + replace]
                dp[i][j] = min(l)
    return dp[rows-1][cols-1]

In [ ]:
def get_dictionary(corpus):
  set_of_words = {}
  for line in corpus :

    for word in line :
      set_of_words.add(word)
  word_dict = []
  for word in set_of_words:
    word_dict.append(word)
  return word_dict

def spell_check(sentence,dictionary,freq,count,phonetic_to_word):
  sentence = clean_sentence(sentence)
  tokens = sentence.split()
  token_to_prob_words = {}
  for i in range(len(tokens)):
    word_to_be_checked = tokens[i]
    contenders = []
    phon_contenders = []
    if word_to_be_checked in dictionary :
      s = calc_phon_score(word_to_be_checked)
      phon_contenders = phonetic_to_word[s]
    min_edit_dist = 10000000000
    for contender in dictionary :
      if abs(len(contender) - len(word_to_be_checked)) > 5 or contender == word_to_be_checked : 
        continue 
      edit_dist = edit_distance(word_to_be_checked,contender)
      if edit_dist == 0 :
        continue
      if edit_dist < min_edit_dist:
        contenders = []
        contenders.append(contender)
        min_edit_dist = edit_dist
      elif edit_dist == min_edit_dist:
        contenders.append(contender) 
    contenders.extend(phon_contenders)
    scores_1 = score(freq,count,tokens[:i+1],i,contenders)
    # scores_2 = {}
    # if word_to_be_checked in dictionary:
    #   scores_2 = score(freq,count,sentence,len(sentence)-1,phon_contenders)
    top_5_scores_1 = dict(itertools.islice(scores_1.items(), 5)) 
    # top_5_scores_2 = {}
    # if len(scores_2) != 0 :
    #   top_5_scores_2 = dict(itertools.islice(scores_2.items(), 5))
    token_to_prob_words[i] = top_5_scores_1
  print(sentence)
  for i in range(len(tokens)-1):
    print('{0} : {1}'.format(tokens[i+1],token_to_prob_words[i+1])) 
  return

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = open('/content/drive/MyDrive/neww_sample.txt','r')
sentences = file.readlines()

In [ ]:
cleaned_data = clean_data(sentences)

In [ ]:
tokens = []
for line in cleaned_data:
  tokens.append(line.split())

In [ ]:
freq,count,dictionary = bigram(tokens)

In [ ]:
phonetic_to_word = get_words_by_phonetic(dictionary)

In [ ]:
phonetic_test_corpus = [
    "कहानी एक मजेदार रोड ड्रिप के रूप में सुरु होती है", # ड्रिप सुरु
    "अब ये सभी होस्ट अपनी बढ़ती उम्र की चुनौतियों के हाथ स्वास्थ्य समस्याओं से जूझ रहे हैं", # होस्ट हाथ 
    "शरीर के भजन का 10 प्रतिशत वार सिर्फ खून का होता है", # भजन वार
    "इसलिए लाल रक्त कोशिकाओं की संख्या पढ़ाने के लिए आयरन वाले फूड्स खाएं", # पढ़ाने
    "तस्‍वीर से दुख रहे इन शख्‍स ने पंजाब से आतंक का खात्‍मा किया।", # दुख
    "सांप का बीस घातक होता है", # बीस 
    "होली त्योहार जल्द ही देश भर में ख़ुशी के साथ मनाया खायेगा", # खायेगा
    "प्रदूषण की दवाव में तेजी से वृद्धि के करने पशु तेजी से अपना वर खो रहे हैं", # करने वर
    "हम अपना अधिकतर समय पढाई पर केंद्रित करने का प्यास करते नजर आते हैं", # प्यास 
    "यह आम पक्का हुआ है", # पक्का 
    "तुम मुझे यह सेब अँधा काटके दे दो", # अँधा
    "राष्ट्रीय सेवक संग ने यह कार्यक्रम आयोजित किया है" # संग
]

In [ ]:
original_test_corpus = [
    "कहानी एक मजेदार रोड ट्रिप के रूप में शुरू होती है",
    "अब ये सभी दोस्त अपनी बढ़ती उम्र की चुनौतियों के साथ स्वास्थ्य समस्याओं से जूझ रहे हैं",
    "शरीर के वजन का 10 प्रतिशत भार सिर्फ खून का होता है",
    "इसलिए लाल रक्त कोशिकाओं की संख्या बढ़ाने के लिए आयरन वाले फूड्स खाएं",
    "तस्‍वीर से दिख रहे इन शख्‍स ने पंजाब से आतंक का खात्‍मा किया।",
    "सांप का विष घातक होता है",
    "होली व्यव्हार जल्द ही देश भर में ख़ुशी के साथ मनाया जायेगा",
    "प्रदूषण के दवाव में तेजी से वृद्धि के कारण पशु तेजी से अपना घर खो रहे हैं",
    "आज के युग में जब हम अपना अधिकतर समय पढाई पर केंद्रित करने का प्रयास करते नजर आते हैं",
    "यह आम पका हुआ है",
    "तुम मुझे यह सेब आधा काटके देदो"
    "राष्ट्रीय सेवक संघ ने यह कार्यक्रम आयोजित किया है",
]

In [ ]:
spelling_mistake_corpus = [
    "कहानी एक मजदार रोड ट्रिप के रूप में शुरू होती है", # मजदार
    "अब ये सभी दोस्त अपनी बढ़ती उम्र की चुनौतयों के साथ स्वास्थ्य समस्याओं से जूझ रहे हैं", # चुनौतयों 
    "इसीलिए लाल रक्त कोशिकाओं की संख्या बराने के लिए आयरन वाले फूड्स खाएं", # इसीलिए बराने
    "तस्‍वीर से दिख रहे इन शख्‍स ने पंजाब से आतक का खात्‍मा किया।", # आतक
    "सांप का विश घातक होता है", # विश  
    "होली तोहार जल्द ही देस भर में ख़ुशी के साथ मनाया जायेगा", # देस व्यव्हार
    "प्रदूषण के दवाब में तेजी से वृद्धि के कारन पशु तेजी से अपना घर खो रहे हैं", # दवाब कारन 
    "आज के यग में जब हम अपना अधकतर समय पढाई पर केंद्रित करने का प्रयास करते नजर आते हैं", # यग अधकतर
    "उनका सेनापति सहीद होगया", # सहीद
    "तुम मुझे यह सेब अध काटके देदो", # अध 
    "राष्ट्रीय सेवक संघ ने यह कार्यकम आयोजित कया है" # कया
]

In [ ]:
for i in range(len(phonetic_test_corpus)):
  spell_check(phonetic_test_corpus[i],dictionary,freq,count,phonetic_to_word)

कहानी एक मजेदार रोड ड्रिप के रूप में सुरु होती है
एक : {'एक': 0.011105857153783733, 'एक्स': 0.001690342316641441, 'टेक': 0.0, 'ऐक': 0.0, 'लेक': 0.0}
मजेदार : {'मजेदार': 0.011146994306357069, 'मज़ेदार': 0.0013872574657360218, 'मज़ेदार': 0.0, 'मजेदाक': 0.0}
रोड : {'रोड': 0.008137361781555889, 'रोज': 0.007078227490774765, 'मोड': 0.0053905958559630485, 'रीड': 0.004895767503388521, 'रोल': 0.0040809752687446105}
ड्रिप : {'ट्रिप': 0.02508928138535676, 'ग्रिप': 0.006940094860640894, 'ड्रिल': 0.00675737378399486, 'ड्रिंक': 0.0017788813366622783, 'ड्रीप': 0.0}
के : {'के': 0.015141111449031233, 'में': 0.01258156537948268, 'से': 0.00951250342332408, 'क्भ्': 0.009180882538226636, 'को': 0.008993204663038108}
रूप : {'रूप': 0.05989559240754836, 'रूस': 0.017478131584720162, 'रेप': 0.013747617770490076, 'रुप': 0.013333875678974061, 'रूट': 0.006561503744904363}
में : {'में': 0.22283296341887315, 'से': 0.18613778913837992, 'के': 0.13060502600421722, 'ने': 0.06160717371212472, 'वे': 0.020837694018009677}
सु

In [ ]:
for i in range(len(spelling_mistake_corpus)):
  spell_check(spelling_mistake_corpus[i],dictionary,freq,count,phonetic_to_word)

कहानी एक मजदार रोड ट्रिप के रूप में शुरू होती है
एक : {'एक': 0.011105857153783733, 'एक्स': 0.001690342316641441, 'टेक': 0.0, 'ऐक': 0.0, 'लेक': 0.0}
मजदार : {'मजेदार': 0.011146994306357069, 'मजदूर': 0.007064311204961585, 'मज़ेदार': 0.0013872574657360218, 'मजदुर': 0.0, 'मतदार': 0.0}
रोड : {'रोज': 0.007078227490774765, 'रोड': 0.00706282075700892, 'रीड': 0.004895767503388521, 'रोल': 0.002364969605037979, 'रोग': 0.002355431997631775}
ट्रिप : {'ट्रिप': 0.01314381932116934, 'ग्रिप': 0.006940094860640894, 'ट्रिक': 0.0026599447668416484, 'ट्रंप': 0.002203692672974139, 'टाइप': 0.001037609753565323}
के : {'के': 0.016372522906114868, 'में': 0.009659563471195363, 'क्भ्': 0.009180882538226636, 'से': 0.008868443022616385, 'की': 0.008756026648660164}
रूप : {'रूप': 0.05989559240754836, 'रूस': 0.017478131584720162, 'रेप': 0.013747617770490076, 'रुप': 0.013333875678974061, 'रूट': 0.006561503744904363}
में : {'में': 0.2213362632715882, 'से': 0.18695137510193197, 'के': 0.13265430619875848, 'ने': 0.06111560